## 데이터프레임의 기본 연산
Series 데이터와 같이 DataFrame에도 `브로드캐스팅`이 적용됩니다.  
예를 들어, 수치 연산은 전체 데이터에 확장 적용됩니다. 

In [ ]:
from pandas import DataFrame

data = [
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가':  90, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  80, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75}, 
]

df = DataFrame(data, index=['20200615', '20200616', '20200617', '20200618'])
df

,시가,고가,저가,종가
20200615,100,110,90,105
20200616,90,112,80,95
20200617,80,115,70,85
20200618,70,80,60,75


df에 덧셈 연산을 적용해 봅시다.

In [ ]:
df+10

,시가,고가,저가,종가
20200615,110,120,100,115
20200616,100,122,90,105
20200617,90,125,80,95
20200618,80,90,70,85


비교 연산 또한 전체 데이터에 적용됩니다. 

In [ ]:
df>10

,시가,고가,저가,종가
20200615,True,True,True,True
20200616,True,True,True,True
20200617,True,True,True,True
20200618,True,True,True,True


비교 연산의 수행 결과 boolean형 데이터가 저장된 DataFrame 객체가 반환됩니다. 

In [ ]:
df[df>100]

,시가,고가,저가,종가
20200615,NaN,110.0,NaN,105.0
20200616,NaN,112.0,NaN,NaN
20200617,NaN,115.0,NaN,NaN
20200618,NaN,NaN,NaN,NaN


In [ ]:
# 시가가 90원이상일 때 그날의 종가를 알고 싶을 때
df[df["시가"] >=90]["종가"]
cond = df["시가"]>=90
df.loc[ cond, ["시가",'종가']]

,시가,종가
20200615,100,105
20200616,90,95


In [ ]:
# 고가와 저가의 차이를 변동폭이라고 정의할 때,
# 변동폭이 15보다 큰 행을 출력
cond = (df["고가"]-df["저가"] ) >20
df.loc[cond]

,시가,고가,저가,종가
20200616,90,112,80,95
20200617,80,115,70,85


Q. 값이 100보다 작은 경우 데이터프레임의 값을 0으로 채워 봅시다. 
- 조건 
- replace
- fillna

판다스는 다양한 메서드를 제공합니다. 

In [ ]:
df = DataFrame(data, index=['20200615', '20200616', '20200617', '20200618'])
df

,시가,고가,저가,종가
20200615,100,110,90,105
20200616,90,112,80,95
20200617,80,115,70,85
20200618,70,80,60,75


`mean` 메서드는 평균을 계산합니다. 

In [ ]:
df.mean( axis=1)

20200615    101.25
20200616     94.25
20200617     87.50
20200618     71.25
dtype: float64

`min` / `max` 함수는 최대, 최소 값을 출력합니다.

In [ ]:
df.max()

시가    100
고가    115
저가     90
종가    105
dtype: int64

Q. 시가에 매수하고 종가에 매도 했을 때의 누적 수익률을 출력하라.

In [ ]:
data = [
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가':  90, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  80, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75}, 
]

df = DataFrame(data, index=['20200615', '20200616', '20200617', '20200618'])
df

,시가,고가,저가,종가
20200615,100,110,90,105
20200616,90,112,80,95
20200617,80,115,70,85
20200618,70,80,60,75


In [ ]:
profit = df["종가"]/df["시가"]
df["수익률"] = profit
df

,시가,고가,저가,종가,수익률
20200615,100,110,90,105,1.050000
20200616,90,112,80,95,1.055556
20200617,80,115,70,85,1.062500
20200618,70,80,60,75,1.071429


Q. 데이터 프레임에서 고가가 가장 높은 날이 날짜를 출력하라.

In [ ]:
data = [
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가':  90, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  80, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75}, 
]

df = DataFrame(data, index=['20200615', '20200616', '20200617', '20200618'])
df


,시가,고가,저가,종가
20200615,100,110,90,105
20200616,90,112,80,95
20200617,80,115,70,85
20200618,70,80,60,75


In [ ]:
cond = df["고가"] ==df["고가"].max()
df[cond].index


Index(['20200617'], dtype='object')

## 데이터프레임 그룹화

복잡한 문제는 잘게 나누어 분석하고, 합쳐나가는 `Split-Apply-Combine` 전략을 사용해서 문제를 해결할 수 있습니다. 판다스가 제공하는 `groupby` 메서드를 사용하면 쉽게 `Split-Apply-Combine`을 사용할 수 있습니다. 
- split: `sex` 칼럼을 기준으로 같은 값을 같는 데이터로 분할합니다. 
- apply: 분할된 각각의 데이터에 mean/min/max/sum 등의 연산을 적용합니다.
- combine: 연산이 적용된 결과를 합쳐 하나의 테이블로 만듭니다. 

<img src="https://i.ibb.co/9Z90Hsy/pandas-1-0.png" width="800" style="float:left" />

다음은 데이터프레임을 사용하 보겠습니다. 

In [38]:
df = DataFrame({
    'sex'    : ['m', 'm', 'w', 'm', 'w'],
    'weight' : [76, 88, 54, 70, 45],
    'height' : [176, 190, 148, 177, 155]        
})
df

,sex,weight,height
0,m,76,176
1,m,88,190
2,w,54,148
3,m,70,177
4,w,45,155


In [47]:
mdf= df [ df["sex"]=="m"]
wdf = df [df["sex"]=="w"]

mdf.mean()
print(mdf)
print(mdf.mean())

  sex  weight  height
0   m      76     176
1   m      88     190
3   m      70     177
weight     78.0
height    181.0
dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [45]:
import pandas as pd


mean_df = pd.concat((mdf.mean(),wdf.mean()),axis=1 )
mean_df.columns=["m","w"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


In [46]:
print(mean_df)

            m      w
weight   78.0   49.5
height  181.0  151.5


`groupby` 메서드를 사용하면 특정 항목을 기준으로 분류할 수 있습니다.  

In [50]:
df_ =df.groupby("sex")
df_.get_group("m")

,sex,weight,height
0,m,76,176
1,m,88,190
3,m,70,177


그룹화한 결과를 확인하기 위해서는 `get_group()` 메서드를 사용해야 합니다.
- 어떤 값을 get 할지 지정해야 함

In [ ]:
df = DataFrame({
    'sex'    : ['m', 'm', 'w', 'm', 'w'],
    'weight' : [76, 88, 54, 70, 45],
    'height' : [176, 190, 148, 177, 155]
})
df

,sex,weight,height
0,m,76,176
1,m,88,190
2,w,54,148
3,m,70,177
4,w,45,155


groupby와 함께 사용할 수 있는 기본 연산은 `mean()` / `min()` / `max()` / `size()`가 있습니다.

In [54]:
df_.size()
df_.max()
df_.min()
df_.mean()

,weight,height
sex,,
m,78.0,181.0
w,49.5,151.5


In [57]:
import numpy as np

`agg` (aggregation) 메서드를 사용해도 같은 기능을 구현할 수 있습니다.

In [59]:
how = {
    "weight": [np.std,np.var]
}

df_.agg(how).loc["m"]

weight  std     9.165151
        var    84.000000
Name: m, dtype: float64

`agg`를 사용하면 각 컬럼 별로 다른 연산을 적용할 수 있습니다. 딕셔너리 형태로 적용할 연산 정보를 전달해야 합니다.

## Reshaping

### Pivot

In [ ]:
data = [
    ["2019", "A", 300, 5],
    ["2019", "B", 200, 4],
    ["2019", "C", 100, 8],
    ["2020", "A", 400, 8],
    ["2020", "B", 230, 3],    
]

columns = ['date', 'item', 'price', 'volume']
df = DataFrame(data=data, columns=columns)
df

,date,item,price,volume
0,2019,A,300,5
1,2019,B,200,4
2,2019,C,100,8
3,2020,A,400,8
4,2020,B,230,3


In [ ]:
df.pivot_table(index="date", columns="item", values="price")

item,A,B,C
date,,,
2019,300.0,200.0,100.0
2020,400.0,230.0,NaN


### Melt
컬럼을 variable과 value로 설정해서 데이터를 긴형태로 변환합니다.   
데이터 cleaning에 유리한 형태로 변환합니다. 

In [ ]:
import numpy as np

data = [
    ["2019", "A", 300, 5],
    ["2019", "B", np.NaN, 4],
    ["2019", "C", 100, np.NaN],
    ["2020", "A", np.NaN, 8],
    ["2020", "B", 230, np.NaN],    
]

columns = ['date', 'item', 'price', 'volume']
df = DataFrame(data=data, columns=columns)
df

,date,item,price,volume
0,2019,A,300.0,5.0
1,2019,B,NaN,4.0
2,2019,C,100.0,NaN
3,2020,A,NaN,8.0
4,2020,B,230.0,NaN


`melt` 메서드를 호출하면 모든 데이터를 `variable`과 `value`로 구분합니다. 

`id_vars` 파라미터를 사용하면 남겨놓을 데이터를 지정할 수 있습니다. 

,date,item,variable,value
0,2019,A,price,300.0
1,2019,B,price,NaN
2,2019,C,price,100.0
3,2020,A,price,NaN
4,2020,B,price,230.0
5,2019,A,volume,5.0
6,2019,B,volume,4.0
7,2019,C,volume,NaN
8,2020,A,volume,8.0
9,2020,B,volume,NaN


In [ ]:
t = df.melt()

In [ ]:
t

,variable,value
0,date,2019
1,date,2019
2,date,2019
3,date,2020
4,date,2020
5,item,A
6,item,B
7,item,C
8,item,A
9,item,B


### Unstack / Stack

In [ ]:
df = DataFrame({
    'city'   : ["서울", "서울", "대구", "대구"],
    'sex'    : ['m', 'w', 'm', 'w'],
    'weight' : [76, 88, 54, 70]
})
df

,city,sex,weight
0,서울,m,76
1,서울,w,88
2,대구,m,54
3,대구,w,70


In [ ]:
temp = df.groupby(['city', 'sex']).max()
temp

weight
city sex        
대구   m        54
     w        70
서울   m        76
     w        88

남자와 여자에 대한 데이터를 구분해서 분석하고 싶을 수 있습니다. 

`unstack` 메서드를 사용하면 하나의 높은 level index를 칼럼으로 변경합니다. 

weight    
sex       m   w
city           
대구       54  70
서울       76  88

인덱스의 레벨을 지정할 수 있습니다. 

weight    
city     대구  서울
sex            
m        54  76
w        70  88

`stack`은 `unstack`의 반대로 동작합니다. 데이터를 길게 쌓아 올립니다. 

## 파일 I/O
대부분의 데이터는 웹상에 존재하거나 파일 형태로 보관됩니다. 판다스는 파일로 데이터를 쓰거나 읽어 올 수 있는 메서드를 제공합니다. 

데이터프레임의 `to_excel` 메서드는 엑셀 파일로 데이터를 저장합니다. 

In [ ]:
df = DataFrame({
    'sex'    : ['m', 'm', 'w', 'm', 'w'],
    'weight' : [76, 88, 54, 70, 45],
    'height' : [176, 190, 148, 177, 155]        
}, index=["영수", "철수", "영자", "말똥", "영희"])
df

,sex,weight,height
영수,m,76,176
철수,m,88,190
영자,w,54,148
말똥,m,70,177
영희,w,45,155


In [ ]:
df.to_excel('test.xlsx')

화면에 아무런 값도 출력되지 않지만, 파일로 데이터가 저장된 것을 확인할 수 있습니다.

<img src="https://i.ibb.co/GFDXBcW/pandas-1-1.png" width="350" style="float:left" />

`sheet_name` 속성을 사용하면 엑셀의 시트탭 이름을 변경할 수 있습니다.

In [ ]:
df.to_excel('test.xlsx', sheet_name = '인적사항')

엑셀에 저장된 데이터도 `pd`의 `read_excel` 메서드를 사용하면 데이터 프레임으로 값을 읽어옵니다.

In [60]:
import pandas as pd

df = pd.read_excel('test.xlsx')
df

FileNotFoundError: ignored

자동으로 부여된 인덱스와 함께 데이터를 읽어 온 것을 확인 할 수 있습니다. 의미있는 이름으로 인덱스를 지정해 보겠습니다.

In [62]:
base_path= "/content/drive/MyDrive/Colab Notebooks/이어드림강의/2022-04-20(유종훈)/3.EDA/"

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
titanic = pd.read_csv(base_path+"test.csv", index_col=0)

추가적인 연산없이 `read_excel`에 `index_col` 옵션을 사용하면 한 번에 인덱스를 설정할 수 있습니다.

In [ ]:
import pandas as pd

df = pd.read_excel('test.xlsx', index_col=0)
df

,sex,weight,height
영수,m,76,176
철수,m,88,190
영자,w,54,148
말똥,m,70,177
영희,w,45,155
